In [1]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [2]:
# add actual data here
prob = [[9.46059663e-01, 7.09195330e-04, 5.32311415e-02],
       [9.04690678e-01, 6.33543191e-04, 9.46757789e-02],
       [8.89115797e-01, 7.38388357e-04, 1.10145814e-01],
       [8.84267593e-01, 1.10228147e-03, 1.14630125e-01],
       [7.46920576e-01, 3.02793284e-04, 2.52776631e-01],
       [8.62265482e-01, 1.54888097e-03, 1.36185637e-01],
       [9.06273425e-01, 3.53315901e-04, 9.33732594e-02],
       [7.02595536e-01, 2.10597217e-04, 2.97193867e-01],
       [9.36170964e-01, 1.82751019e-03, 6.20015254e-02],
       [7.70851596e-01, 3.16971482e-04, 2.28831433e-01],
       [9.40920924e-01, 8.31554474e-04, 5.82475219e-02],
       [7.85572392e-01, 8.95049951e-04, 2.13532558e-01],
       [8.08803302e-01, 4.08858457e-04, 1.90787840e-01],
       [9.51687212e-01, 1.73266602e-04, 4.81395210e-02],
       [8.98711445e-01, 5.73961822e-04, 1.00714593e-01],
       [7.85426057e-01, 9.65735820e-04, 2.13608208e-01],
       [9.85815115e-01, 1.06279075e-04, 1.40786060e-02],
       [8.73873998e-01, 2.29608186e-03, 1.23829920e-01],
       [9.19966136e-01, 3.37531100e-04, 7.96963325e-02],
       [8.56100038e-01, 5.51187702e-04, 1.43348774e-01],
       [7.43772225e-01, 4.43218949e-04, 2.55784556e-01],
       [8.51217909e-01, 5.32479430e-04, 1.48249612e-01],
       [8.43526699e-01, 4.33627348e-04, 1.56039674e-01],
       [8.96392380e-01, 7.25157223e-04, 1.02882463e-01],
       [9.48399670e-01, 1.35747296e-03, 5.02428572e-02],
       [9.01036657e-01, 4.02319870e-04, 9.85610236e-02],
       [8.61103532e-01, 7.36841369e-04, 1.38159627e-01],
       [9.17327689e-01, 4.64848194e-04, 8.22074630e-02],
       [8.99079696e-01, 9.72302369e-04, 9.99480017e-02],
       [8.62422033e-01, 7.28236790e-04, 1.36849730e-01],
       [6.65300240e-01, 5.39423516e-04, 3.34160336e-01],
       [8.16632446e-01, 1.02315422e-04, 1.83265239e-01],
       [9.16126491e-01, 1.03179796e-04, 8.37703294e-02],
       [9.12339314e-01, 2.79704625e-04, 8.73809814e-02],
       [9.23874109e-01, 8.92111477e-04, 7.52337791e-02],
       [9.21860358e-01, 2.68180171e-04, 7.78714616e-02],
       [7.30468827e-01, 3.06628346e-04, 2.69224545e-01],
       [6.23061950e-01, 8.98223514e-05, 3.76848227e-01],
       [9.06171473e-01, 2.29155464e-04, 9.35993714e-02],
       [6.60809521e-01, 8.38397835e-05, 3.39106639e-01],
       [9.65865166e-01, 1.94724888e-04, 3.39401092e-02],
       [9.76708312e-01, 2.66362543e-04, 2.30253254e-02],
       [8.64652762e-01, 5.88198566e-04, 1.34759040e-01],
       [9.10666186e-01, 2.15975347e-04, 8.91178382e-02],
       [9.03383858e-01, 3.84001522e-04, 9.62321405e-02],
       [9.25664382e-01, 3.90480247e-04, 7.39451374e-02],
       [9.32914171e-01, 3.20571281e-04, 6.67652575e-02],
       [9.03508449e-01, 7.24177840e-04, 9.57673733e-02]]

48-element Vector{Vector{Float64}}:
 [0.946059663, 0.00070919533, 0.0532311415]
 [0.904690678, 0.000633543191, 0.0946757789]
 [0.889115797, 0.000738388357, 0.110145814]
 [0.884267593, 0.00110228147, 0.114630125]
 [0.746920576, 0.000302793284, 0.252776631]
 [0.862265482, 0.00154888097, 0.136185637]
 [0.906273425, 0.000353315901, 0.0933732594]
 [0.702595536, 0.000210597217, 0.297193867]
 [0.936170964, 0.00182751019, 0.0620015254]
 [0.770851596, 0.000316971482, 0.228831433]
 [0.940920924, 0.000831554474, 0.0582475219]
 [0.785572392, 0.000895049951, 0.213532558]
 [0.808803302, 0.000408858457, 0.19078784]
 ⋮
 [0.730468827, 0.000306628346, 0.269224545]
 [0.62306195, 8.98223514e-5, 0.376848227]
 [0.906171473, 0.000229155464, 0.0935993714]
 [0.660809521, 8.38397835e-5, 0.339106639]
 [0.965865166, 0.000194724888, 0.0339401092]
 [0.976708312, 0.000266362543, 0.0230253254]
 [0.864652762, 0.000588198566, 0.13475904]
 [0.910666186, 0.000215975347, 0.0891178382]
 [0.903383858, 0.000384001522, 0.0962

In [17]:
# # # DUMMY DATA JUST TO TEST OUT FORMULATION
# # set parameters
# Random.seed!(123)
# n = 3
# m = 48
# probabilities = zeros(m,n)
# returns_on_100 = zeros(m,n)

# # fill in probabilities by generating 3 numbers between 0 and 1, and sum them to 1
# for i in 1:m
#     p = rand(n)
#     p = p ./ sum(p)
#     probabilities[i,:] = p
# end

# # generate 3 random numbers between 1 and 5 for each row of returns_on_100
# for i in 1:m
#     returns_on_100[i,:] = rand(n) .* 5
# end

## ____________________________________________________________________

In [31]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*(1/100)*R[i,1]*x[i,1] + 
                               P[i,2]*(1/100)*R[i,2]*x[i,2] + 
                               P[i,3]*(1/100)*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B*z[i,j] for all i and j
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [32]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(probabilities, returns_on_100, 100000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 194 rows, 288 columns and 720 nonzeros
Model fingerprint: 0x1d77f1af
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%

User-callback calls 22, time in user-callback 0.00 sec
